<img src="./src/logo.png" width="250">

**Baustein:** Klassifikation  $\rightarrow$ **Subbaustein:** Grundlagen und $k$-Nearest Neighbour $\rightarrow$ **Übungsserie**

**Version:** 1.0, **Lizenz:** <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">CC BY-NC-ND 4.0</a>

***

# Klassifikation 1: Grundlagen und $k$-Nearest Neighbour

---
## Importieren der notwendigen Python-Bibliotheken

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import *
import seaborn as sns
from seaborn import pairplot
import matplotlib.pyplot as plt
from collections import Counter

%run src/setup.ipynb

---
## Importieren der Daten
#### Aufgabe 1: Passen Sie den Importierbefehl so an, dass der gewünschte Datensatz in der Variable ```df``` gespeichert ist.
Hierbei hilft die Bibliothek `pandas`, die es erlaubt, zum Beispiel eine `.csv`-Datei als **Dataframe-Objekt** einzulesen. Andere Formate, die mit `pandas` eingelesen werden können sind z.B. `.xlsx`, `.hdf5` oder `.json`.


Verfügbare Datensätze sind:

| Art                                                                                                                  | Name                  |
|----------------------------------------------------------------------------------------------------------------------|-----------------------|
| [Pflanzempfehlungen](https://www.kaggle.com/datasets/chitrakumari25/smart-agricultural-production-optimizing-engine) | pflanzempfehlung      |
| [Herzinfarkt-Risiko](https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset)    | herzinfarkt           |
| [Kickstarter-Projekte](https://www.kaggle.com/datasets/ulrikthygepedersen/kickstarter-projects)                      | kickstarter           |
| [Krebs-Klassifikation](https://www.kaggle.com/datasets/erdemtaha/cancer-data)                                        | krebs                 |
| [Glass-Identifikation](https://www.kaggle.com/datasets/prashant111/glass-identification-dataset)                     | glas                  |
| [Kundenpersönlichkeits-Analyse](https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis)           | kundenpersoenlichkeit |
| [Pinguin-Klassifikation](https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data)     | penguins              |


In [ ]:
datensatz = 
PATH = './data/' # Setzen eines (relativen) Pfades zum Verzeichnis, das den Datensatz enthält
df = pd.read_csv(PATH + datensatz + '_preprocessed.csv') # Laden des Trainings-Datensatzes

Zum Verschaffen eines Überblicks wird der Datensatz einmal ausgegeben.

In [ ]:
df

---
## Klassen
Machen Sie sich mit dem Datensatz vertraut.
#### Aufgabe 2: Was ist die Spalte mit den Klassen? Geben Sie den Namen der Spalte mit den Klassen in der zugehörigen Variable `klasse` an. Halten Sie fest, wie viele Klassen es gibt und welche es sind.

Antwort:



In [ ]:
klasse = 'Spaltenname'
print(df[klasse].unique()) # Ausgabe aller einzigartigen Werte innerhalb der definierten Spalte der Klasse

Zur Übersichtlichkeit sollen zuerst einmal zwei Klassen voneinander unterschieden werden können.
#### Aufgabe 3: Suchen Sie sich diese beiden Klassen aus und ändern sie die Werte `klasse1` und `klasse2` dementsprechend.

Die restlichen Klassen werden aus dem **Dataframe-Objekt** entfernt.

In [ ]:
klasse1 = 'klasse1'
klasse2 = 'klasse2'

df_reduced = df[df[klasse].isin([klasse1, klasse2])] # Entfernen aller Reihen im Datensatz, die nicht der vorgegebenen Klassenwerte entsprechen

df_reduced

#### Aufgabe 4: Vergleichen Sie bei der Ausgabe von `df_reduced` die Anzahl der Reihen mit der von `df`.

(Manche Datensätze enthalten nur 2 Klassen.)

Antwort:


---
## Merkmale
Nutzen Sie zum Visualisieren der Daten den **Pairplot** und die **Korrelationsmatrix**, um aussagekräftige Merkmale zu identifizieren. 


(Dieser Schritt kann in der Ausführung länger dauern -- je nach Größe des Datensatzes und Anzahl der Merkmale)

In [ ]:
plt.figure()
pairplot(df_reduced, hue=klasse, plot_kws={'alpha': 0.5})
plt.show()

Um die Korrelationen berechnen zu können müssen metrische/numerische Werte vorliegen. Konvertieren Sie daher die kategorischen Merkmalsausprägungen der Klassenspalte.

In [ ]:
df_heatmap = df_reduced.copy() # erstelle eine unabhängige Kopie des dataframes
df_heatmap[klasse] = df_heatmap[klasse].map({klasse1: 1, klasse2: 0})  # Zuteilung numerischer Werte der kategorischen Merkmalsausprägungen, sinnvolle Reihenfolge beachten!

In [ ]:
plt.figure()
plt.subplots(figsize=(15,13))
sns.heatmap(df_heatmap.corr(), annot=True, cmap="Blues") 
plt.title("Korrelation zwischen den Merkmalen")
plt.show()

#### Aufgabe 4: Was gibt es für Merkmale? Sind alle sinnvoll? Entdecken Sie auffällige Korrelationen zwischen den Merkmalen? Geben Sie diese ggf. an.

Antwort:


#### Aufgabe 5: Beschränken Sie sich für die weiteren Schritte auf zwei Merkmale. Wieso haben Sie sich für diese beiden Merkmale entschieden? Geben Sie die gewählten Merkmale in den zugehörigen Variablen `merkmal1` und `merkmal2` an. Vergleichen Sie bei der Ausgabe von `df_reduced` die Anzahl der Spalten mit der von `df`. 

Zur Visualisierung später ist es sinnvoll vorerst Merkmale mit **kontinuierlichen Daten** zu wählen.

Antwort:


In [ ]:
merkmal1 = 'merkmal1'
merkmal2 = 'merkmal2'

df_reduced = df_reduced[[merkmal1, merkmal2, klasse]] # Reduzieren des Dataframes auf die vorgegebenen Spalten
print(df_reduced)

---
## Verteilung der Klassen
Eine ungleichmäßige Verteilung der Häufigkeit der Klassen kann zu falschen Klassifikationen führen, da die häufig vorkommende Klasse bevorzugt klassifiziert wird. Daher ist es wichtig sich vor der Klassifikation ein Bild über die Verteilung zu machen. 

#### Aufgabe 6: Wie verhält sich die Verteilung für den von Ihnen gewählten Datensatz?

Antwort:


In [ ]:
plt.figure()
sns.countplot(data=df_reduced,x=klasse)
plt.show()

---
## Aufteilen des Datensatzes in Trainings- und Testdaten
Das **Dataframe-Objekt** wird aufgeteilt in zwei Variablen -- zum einen in die Variable `X` für die Merkmale und zum anderen in die Variable `Y` für die Klassen-Labels. Zudem wird weiterhin ein Trainings- und ein Testdatensatz erstellt. Anhand des Trainingsdatensatzes soll der Klassifikator "trainiert" werden. Die Testdaten sollen klassifiziert werden, um zu überprüfen, wie gut der Klassifikator bei neuen Daten performt. Mit dem Parameter `test_size` können Sie die Größe des Testdatensatzes beeinflussen: $0.2$ steht für eine Größe von 20% des Gesamtdatensatzes. 

#### Aufgabe 7: Verändern Sie die Aufteilung der Daten und beobachten Sie wie sich das Verhältnis ändert. Erklären Sie die Dimensionen der 4 Variablen.

Antwort:



In [ ]:
Y = df_reduced[klasse] # y enthält alle Label für alle Datenpunkte
X = df_reduced.drop(columns=[klasse]) # X enthält alle Merkmale für alle Datenpunkte
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42) # Split der Daten mit der Verteilung von test_size mit Shuffeln
print('Dimension X_train: ', X_train.shape)
print('Dimension Y_train: ', Y_train.shape)
print('Dimension X_test: ', X_test.shape)
print('Dimension Y_test: ', Y_test.shape)

---
## Festlegen der Parameter
#### Aufgabe 8: Legen sie im Nachfolgenden die nachfolgenden Parameter fest.
- `k` (wie viele benachbarte Punkte sollen in die Klassifikationsentscheidung mit einfließen) und
- `p` (welche Distanznorm soll genutzt werden, um die nächsten Nachbarn zu finden)
    - $p=1$ entspricht der Manhatten-Distanz
    - $p=2$ entspricht der Euklidischen-Distanz


In [ ]:
k = 
p = 

---
## Festlegen eines Testpunkts

#### Aufgabe 9: Welche Merkmalwerte soll der zu klassifizierende Testpunkt `test_pt` haben?
Schauen Sie im Internet nach entsprechenden Werten oder denken Sie sich einen Testpunkt aus. Dieser Testpunkt soll im Nachfolgenden klassifiziert werden.
Um das Ergebnis überprüfen zu können, sollten Sie, falls Sie keine Aussage über die richtige Klasse des Testpunktes treffen können, einen Testpunkt aus dem Testdatensatz `X_test` zusammen mit dem dazugehörigen Label aus `Y_test` auswählen.

In [ ]:
test_punkt = pd.DataFrame([{merkmal1: XXX, merkmal2: XXX}])
print(test_punkt)

---
## Visualisierung Testpunkt
Lassen Sie sich in den **Scatterplot** den Testpunkt mit darstellen. 

#### Aufgabe 10: Wie würden Sie den Testpunkt rein visuell/durch ggf. Expertise auf dem Bereich klassifizieren? Wie sicher sind Sie sich hierbei?

Aufgabe: 

In [ ]:
plt.figure()
sns.scatterplot(data=X_train, x=merkmal1, y=merkmal2, hue=Y_train, palette=[ETIT,MTBT])
plt.plot(test_punkt[merkmal1], test_punkt[merkmal2], 'o', color=BW, label='Testpunkt')
plt.show()

---
## $k$-Nearest Neighbour Klassifikator
Mithilfe der Klasse [`KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) von `sklearn` wird der Klassifikator für die Trainingsdaten (`X_train`, `Y_train`) erstellt.

In [ ]:
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train, Y_train)

Das Verfahren lässt sich aber auch relativ einfach selbst implementieren. Schauen Sie sich die einzelnen Schritte an und vergleichen Sie diese mit denen aus der Vorlesung. Berechnen Sie die Distanz zu einem Trainingsdatenpunkt selbst und vergleichen Sie dieses Ergebnis mit dem von Python berechneten. 

#### Aufgabe 11: Vervollständigen Sie hierfür den bereits vorimplementierten Algorithmus zur Berechnung der Minkowski-Distanz in Zeile 9. 
Zur Erinnerung die Formel der Minkowski-Distanz:
$d\left(a, b\right)=\left(\displaystyle\sum_{i=1}^D\left|a_{i}-b_{i}\right|^p\right)^{1 / p}$

In [ ]:
# Berechnung der Distanz zwischen den Punkten a und b
def minkowski_distanz(a, b, p=1):
    # Speichern der Dimensionen (Anzahl an Merkmalen) von Punkt a
    dimension = len(a)
    # Initalisiere die Variabel distanz auf 0
    distanz = 0
    # Berechnung der Minkoswki Distanz mithilfe des festgelegten Parameters p
    for i in range(dimension):
        distanz = distanz + abs(????)^p #ergänzen
    return distanz**(1/p)

# Berechnung der Distanzen zwischen dem Testpunkt test_pt und allen anderen Trainingspunkten X
distanzen = []
for j in X_train.index:
    distanzen.append(minkowski_distanz(test_punkt.values[0], X_train.loc[j], p))

df_dists = pd.DataFrame(data=distanzen, index=X_train.index, columns=['Distanz'])
print(df_dists)

In [ ]:
# Finden der k-nächsten Nachbarn
df_dists = df_dists.sort_values(by=['Distanz'], axis=0)[0:k] # Sortieren der k-nächsten Distanzen nach Größe
df_nn = df_dists.join(df_reduced) # Integrieren der Daten aus dem ursprünglichen Dataframe zu den Distanzen
print(df_nn)

In [ ]:
# Zählen der Labels der nächsten Nachbarn
counter = Counter(Y_train[df_nn.index])
print(counter)

---
## Klassifikation des Testpunkts
Im Folgenden wird der Testpunkt einmal mit dem selbst implementierten Verfahren und unter Zuhilfenahme des `sklearn` Pakets klassifiziert. Für Letzteres ermöglicht die Methode `predict` die Klassifikation mithilfe des eben erstellten Klassifikators.

In [ ]:
# Finden des am meisten vorkommenden Labels
label_testpt = counter.most_common()[0][0]
print('Vorhersage für Testpunkt mit eigenem implementieren Verfahren:\n', label_testpt)

In [ ]:
# Vorhersage mit scikit-learn
label_testpt_sklearn = clf.predict(???)[0] #Ändern
print('Vorhersage für Testpunkt mithilfe des scikit-learn-Pakets:\n', label_testpt_sklearn)

---
## Klassifikation eines ganzen Testdatensatzes
Um die Klassifikationsgenauigkeit des erstellten $k$-NN Klassifikators zu erfassen, klassifizieren Sie im Folgenden den gesamten Testdatensatz und lassen Sie sich dann die **Accuracy** ausgeben. 
#### Aufgabe 12: Haben Sie diese Accuracy erwartet? Begründen Sie.

Antwort: 

In [ ]:
predictions = clf.predict(???) #Ändern
print('Accuracy: ', accuracy_score(Y_test, predictions))

#### Aufgabe 13: Ist der Wert der Accuracy als Klassifikationsgenauigkeit aussagekräftig? Begründen Sie ebenfalls anhand der **Confusion Matrix**.

Antwort:

In [ ]:
plt.figure()
cm = confusion_matrix(Y_test, predictions)
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Vorhersage von "' + klasse + '"')
plt.ylabel('Wahrer Wert von "' + klasse + '"')

plt.show()

#### Aufgabe 14: Entscheiden Sie, welche Metrik Sie für die Klassifikationsgenauigkeit ebenfalls berechnen lassen wollen. Recherchieren Sie dafür auf der Dokumentation der Python-Bibliothek `scikit-learn` (sklearn) welche Klassifikationsgenauigkeit-Metriken es gibt. Wählen Sie eine aus, begründen Sie Ihre Wahl und lassen Sie sich diese berechnen. 

Beispielhaft ist dies bereits erfüllt für den **F1-Score**. Hier ist es ebenfalls notwendig das `pos_label` anzugeben. Hierbei handelt es sich um die Klasse, die ggf. unterrepräsentiert ist, also weniger Datenpunkte enthält als die andere.

Antwort:

In [ ]:
positiv_label = df_reduced[klasse].value_counts().index[-1] # entspricht der Klasse, die im Datensatz weniger vertreten ist
print('F1-Score: ', f1_score(Y_test, predictions, pos_label=positiv_label))

---
## Parameter-Wahl
#### Aufgabe 15: Verändern Sie in der nachfolgenden Visualisierung die Parameter `k` und `p` (Distanznorm). Gibt es einen Wert für `k` bei dem die Klassifikation eine falsche Vorhersage trifft? Wenn ja, woran könnte dies liegen? Was kann bei der Wahl eines geraden Werts für `k` passieren? Ändern Sie auch die Art der Vorverarbeitung und die Daten des Testpunkts. Was passiert, wenn der Testpunkt außerhalb des Bereichs der Trainingsdaten liegt? Beschreiben Sie die Auswirkung der Vorverarbeitung auf die nächsten Nachbarn des Testpunkts.

Antwort:

In [ ]:
%matplotlib widget
exec(open('src/interact_kNN_widget.py').read())
# farbe von boundary display anpassen; klassenzuordnung

In [ ]:
# Visualisierung ohne Widgets
# from src.interact_kNN import *
# nearest_neighbour(Distanznorm='Euklidisch', Vorverarbeitung='keine', Testpunkt_Merkmal1=60, Testpunkt_Merkmal2=150, k=10, df_reduced=df_reduced, merkmal1=merkmal1, merkmal2=merkmal2, klasse=klasse, klasse1=klasse1, klasse2=klasse2)

---
## Merkmale und Einfluss der Vorverabeitung
Im Folgenden wird der Datensatz neu geladen und in der Variable `x` dieses Mal alle Merkmale abgespeichert.

#### Aufgabe 16: Untersuchen Sie wie sich die gewählte repräsentative Klassifikationsgenauigkeit(en) mit allen Merkmalen verändert.
Verändern Sie den Code so, dass Ihre gewünsche Klassifikationsgenauigkeit ausgegeben wird.
Normalisieren Sie daraufhin die Werte der Merkmale und schauen Sie wie sich die Klassifikationsgenauigkeit verändert. Begründen Sie diese Veränderung.

Antwort: 

In [ ]:
# Auswählen der zwei definierten Klassen für den Dataframe
df_allFeatures = df[df[klasse].isin([klasse1, klasse2])]

# Split des Datensatzes in Merkmale und Label
X = df_allFeatures.drop(columns=[klasse]) # X enthält alle Merkmale für alle Datenpunkte
Y = df_allFeatures[klasse] # y enthält alle Label für alle Datenpunkte

############################
# Normalisieren der Merkmale MinMaxScaler(Normalisierung) oder StandardScaler(Standardisierung)
#scaler = MinMaxScaler()
#X = scaler.fit_transform(X)
############################

# Split des Datensatzes in Trainings- und Testdaten
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)

# Übergeben der Trainingsdaten an den erstellten kNN-KLassifikator
clf = KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train, Y_train)

# Vorhersage des Testdatensatzes
predictions = clf.predict(X_test)

# Ausgabe der gewählten Klassifikationsgenauigkeit
positiv_label = df_allFeatures[klasse].value_counts().index[-1]
print('F1-Score: ', f1_score(Y_test, predictions, pos_label=positiv_label))
??????

---
## Auswirkung des Parameters $\boldsymbol{k}$
#### Aufgabe 17: Nutzen Sie die for-Schleife, um sich für $1<k<k_{max}$ die Klassifikationsgenauigkeit Ihrer Wahl ausgeben zu lassen. Speichern Sie dafür den Wert der Klassifikationsmetrik für jedes `k` in der Variable `score`.

In [ ]:
genauigkeit = []
k_max = ???
for k in range (1, ???):
    clf = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
    clf.fit(X_train, Y_train)
    predictions = clf.predict(X_test)
    score = accuracy_score(Y_test, predictions)
    positiv_label = df[klasse].value_counts().index[-1]
    score = f1_score(Y_test, predictions, pos_label=positiv_label)
    genauigkeit.append(score)

#### Aufgabe 18: Stellen Sie das Ergebnis in einer Grafik dar. Wo liegt der beste Wert für `k`? Was kann bei einem geraden Wert für `k` passieren? Begründen Sie den Verlauf der Grafik.

Antwort:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(range(1, k_max), genauigkeit, color=ETIT,
         linestyle='dashed', marker='o',
         markerfacecolor=MTBT, markersize=10)

plt.title('Klassifikationsgenauigkit in Abhängigkeit von $k$')
plt.xlabel('k')
plt.ylabel('Klassifikationsgenauigkeit der selbstgewählten Metrik')

#### Aufgabe 19: Speichern Sie den besten Wert für $k$ in der Variable `k_opt`, um damit weiterzuarbeiten.
($k$ hat dann für diesen Train-Test-Split und diese Vorverarbeitung den besten Wert. Bei einem erneuten Shuffeln der Daten kann sich jedoch ein anderer optimaler Wert für $k$ ergeben.)

In [ ]:
k_opt = genauigkeit.index(max(genauigkeit)) + 1
print(k_opt)

#### Aufgabe 20: Lassen Sie die Klassifikationsgenauigkeit mit dem angepasst `k_opt`, den normalisierten Daten und allen Merkmalen erneut berechnen. Hat sich die Genauigkeit verbessert? Begründen Sie.

Antwort:

In [ ]:
# Übergeben der Trainingsdaten an den erstellten kNN-KLassifikator
clf = KNeighborsClassifier(n_neighbors=???)
clf.fit(X_train, Y_train)

# Vorhersage des Testdatensatzes
predictions = clf.predict(X_test)

# Ausgabe der gewählten Klassifikationsgenauigkeit
positiv_label = df_allFeatures[klasse].value_counts().index[-1]
print('Accuracy: ', accuracy_score(Y_test, predictions))
print('F1-Score: ', f1_score(Y_test, predictions, pos_label=positiv_label))

#### Aufgabe 21: Wie geeignet war Ihr Datensatz für eine Klassifikation mit $k$NN? Woran machen Sie die Einschätzung fest? Tauschen Sie sich mit anderen Studierenden über die unterschiedlichen Datensätze aus.

Antwort:

#### Zusatzaufgabe: Verändern Sie den Code am Anfang so, dass andere Merkmale untersucht werden. Können Sie so Aussagen über die Wichtigkeit verschiedener Merkmale treffen?

Antwort: 

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Die Übungsserie begleitend zum AI4ALL-Kurs</span> der <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">EAH Jena</span> ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Namensnennung - Nicht kommerziell - Keine Bearbeitungen 4.0 International Lizenz</a>.

Der AI4ALL-Kurs entsteht im Rahmen des Projekts MoVeKI2EAH. Das Projekt MoVeKI2EAH wird durch das BMBF (Bundesministerium für Bildung und Forschung) und den Freistaat Thüringen im Rahmen der Bund-Länder-Initiative zur Förderung von Künstlicher Intelligenz in der Hochschulbildung gefördert (12/2021 bis 11/2025, Föderkennzeichen 16DHBKI081).